In [1]:
import numpy as np
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

6. (5 баллов) Камера повернута относительно мировой системы координат на 45
градусов вокруг оси z и сдвинута на 10 вдоль оси z мировой системы координат.
Внутренние параметры камеры: fx=fy=400, cx=960, cy=540.<br>
Написать программу, которая выводит на экран матрицу проекции камеры и целочисленные
координаты пикселя на изображении, соответствующего трехмерной точке с
координатами (10, -10, 100) в мировой системе координат.

In [2]:
fx = fy = 400
cx = 960
cy = 540
angle_x = 0
angle_y = 0
angle_z = 45

theta = np.radians(45)
shift = np.array([0, 0, 10])
world = np.array([10, -10, 100])

In [3]:
def rotation_matrix(theta1, theta2, theta3):
    w = np.radians(theta1)
    phi = np.radians(theta2)
    k = np.radians(theta3)

    R_x = np.array([[1, 0, 0],
                    [0, np.cos(w), -np.sin(w)],
                    [0, np.sin(w), np.cos(w)]])
    R_y = np.array([[np.cos(phi), 0, np.sin(phi)],
                    [0, 1, 0],
                    [-np.sin(phi), 0, np.cos(phi)]])
    R_z = np.array([[np.cos(k), -np.sin(k), 0],
                    [np.sin(k), np.cos(k), 0],
                    [0, 0, 1]])
    return R_x @ R_y @ R_z

In [4]:
R = rotation_matrix(angle_x,angle_y,angle_z)

In [5]:
K = np.array([[fx, 0, cx],
             [0, fy, cy],
             [0, 0, 1]])

In [6]:
P = np.dot(K, np.insert(R, 3, shift, axis=1))
print("Проекционная матрица:")
print(np.round(P).astype(int))

Проекционная матрица:
[[ 283 -283  960 9600]
 [ 283  283  540 5400]
 [   0    0    1   10]]


In [7]:
point_camera = R @ world
u = (fx * point_camera[0] / point_camera[2]) + cx
v = (fy * point_camera[1] / point_camera[2]) + cy

u_int = int(u)
v_int = int(v)

print(f"Координаты пикселя на изображении:")
print("x:", u_int)
print("y:", v_int)

Координаты пикселя на изображении:
x: 1016
y: 540


7. (5 баллов) Написать генератор пар двумерных точек для теста алгоритма,
удовлетворяющих заданному заранее преобразованию гомографии H0. Используя
прямое линейное преобразование (DLT) и SVD, написать программу, которая
принимает на вход множество пар двумерных точек, и находит соответствующую
этим парам матрицу гомографии H. Распечатать матрицы H0 и H

In [8]:
def generate_pairs(num_pairs, H0):
    points_source = np.random.rand(num_pairs, 2)
    points_source_homogeneous = np.hstack((points_source, np.ones((num_pairs, 1))))
    points_target_homogeneous = np.dot(H0, points_source_homogeneous.T).T
    points_target = points_target_homogeneous[:, :2] / points_target_homogeneous[:, 2:]
    return points_source, points_target

def find_homography(points1, points2):
    points_source_h = np.hstack((points1, np.ones((points1.shape[0], 1))))
    points_target_h = np.hstack((points2, np.ones((points2.shape[0], 1))))
    A = []
    for i in range(len(points_source_h)):
        x, y, z = points_source_h[i]
        u, v, z = points_target_h[i]
        A.append([-x, -y, -1, 0, 0, 0, u*x, u*y, u])
        A.append([0, 0, 0, -x, -y, -1, v*x, v*y, v])
    A = np.array(A)

    U, S, V = np.linalg.svd(A)
    H = V[-1].reshape(3, 3)
    H /= H[2, 2]  # Нормализуем гомографию
    return H


In [9]:
H0 = np.array([[1., 2., 3.],
               [4., 5., 6.],
               [7., 8., 9.]])
num_pairs = 8
points1, points2 = generate_pairs(num_pairs, H0)

In [10]:
H = find_homography(points1, points2)

print("Матрица гомографии H0:")
print(H0)
print("Найденная матрица гомографии H:")
print(H)

Матрица гомографии H0:
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
Найденная матрица гомографии H:
[[0.11111111 0.22222222 0.33333333]
 [0.44444444 0.55555556 0.66666667]
 [0.77777778 0.88888889 1.        ]]


8. (5 баллов) Модифицировать генератор из Задачи 7 так, чтобы он выдавал n=1000
соответствий, из которых 30% соответствуют заданному заранее преобразованию
гомографии, а остальные взяты случайно (преобразованию гомографии не
удовлетворяют). Реализовать нахождение преобразования гомографии H1,
используя DLT и RANSAC. Распечатать H0 и H1.

In [11]:
H0 = np.array([[1., 2., 3.],
               [4., 5., 6.],
               [7., 8., 9.]])
n = 1000

In [12]:
def generate_homography_points_with_noise(H0, n, homography_percent = 0.3):
    homography_n = int(n*homography_percent)
    print("Соответствие гомографии:", homography_n)
    print("Случайное соответствие:", n - homography_n)

    points_source = np.random.rand(n, 2)
    points_target = np.array([np.dot(H0, np.insert(points_source[i], 2, 1, axis=0)) for i in range(homography_n)])
    points_target = (points_target/points_target[:,2:])[:,:2]
    points_noise = np.random.rand(n - homography_n, 2) 
    points_target = np.concatenate([points_noise, points_target])
    np.random.shuffle(points_source)
    np.random.shuffle(points_target)

    return points_source, points_target

In [13]:
points_source, points_target = generate_homography_points_with_noise(H0, n, 0.3)

Соответствие гомографии: 300
Случайное соответствие: 700


In [14]:
def DLT(points_source, points_target):
    A = []
    for src, dst in zip(points_source, points_target):
        x, y = src
        u, v = dst
        A.append([-x, -y, -1, 0, 0, 0, x*u, y*u, u])
        A.append([0, 0, 0, -x, -y, -1, x*v, y*v, v])
    A = np.array(A)
    U, S, V = np.linalg.svd(A)
    H = V[-1].reshape(3, 3)
    return H

def geometric_distance(point_source, point_target, H):
    estimatep2 = np.dot(H, np.array(list(point_source) + [1]).reshape((3,1)))
    if estimatep2[2] != 0:
        estimatep2 = estimatep2/estimatep2[2]

    error = np.dot(H, np.array(list(point_target) + [1]).reshape((3,1))) - estimatep2
    return np.linalg.norm(error)

def RANSAC(points_source, points_target, n, max_iter = 500, threshold=0.5):
    max_inliers = 0
    best_H = None
    curr_idx = np.zeros(4)
    curr_points_source = np.zeros(4, dtype=object)
    curr_points_target = np.zeros(4, dtype=object)
    for i in range(max_iter):
        iscolinear = True
        while(iscolinear):
            iscolinear = False
            for j in range(4):
                curr_idx[j] = random.randint(0, n-1)
                for i in range(j):
                    if (curr_idx[j] == curr_idx[i]):
                        iscolinear = True
                        break
                if (iscolinear):
                    break
                curr_points_source[j] = tuple(points_source[int(curr_idx[j])])
                curr_points_target[j] = tuple(points_target[int(curr_idx[j])])

        H1 = DLT(curr_points_source, curr_points_target)

        inliers = 0
        for j in range(n):
            d = geometric_distance(points_source[j], points_target[j], H1)
            if d < threshold:
                inliers += 1
        if inliers > max_inliers:
            max_inliers = inliers
            best_H = H1
    return best_H, max_inliers

In [15]:
H1, status = RANSAC(points_source, points_target, n)

In [16]:
print("Матрица H0:")
print(H0)
print("Матрица H1 (RANSAC):")
print(H1)

Матрица H0:
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
Матрица H1 (RANSAC):
[[0.08396836 0.04828665 0.20289621]
 [0.2662123  0.19188694 0.40360637]
 [0.44845624 0.33548723 0.60431653]]


9. (5 баллов) Два изображения одной и той же сцены сделаны одной и той же
камерой с такой же матрицей внутренних параметров, как в Задаче 6. Второй
снимок сделан после поворота камеры на 30 градусов вокруг оси x относительно
начала координат системы отсчета, связанной с камерой (без сдвига). Найти
матрицу гомографии между двумя изображениями.

In [17]:
fx = fy = 400
cx = 960
cy = 540
angle_x = 30
angle_y = 0
angle_z = 0

In [18]:
theta = np.radians
R = rotation_matrix(angle_x, angle_y, angle_z)
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0, 0, 1]])

H = K @ R @ np.linalg.inv(K)
H = H / H[2,2]

print("Матрица гомографии H между изображениями:")
print(H)

Матрица гомографии H между изображениями:
[[ 5.23490583e+00  6.28188700e+00 -4.06550960e+03]
 [ 0.00000000e+00  8.06712287e+00 -2.95510434e+03]
 [ 0.00000000e+00  6.54363229e-03  1.00000000e+00]]
